<a href="https://colab.research.google.com/github/ficorrea/data_science/blob/main/teste_ab/preparacao_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Teste AB -> gerenciamento de risco retorno

Preparação dos dados
* Coleta
    * Análise descritiva
    * Verificação de faltantes
    * Conferir flags do teste A
* Preparação
    * Amostragem aleatória dos grupos Controle e Tratamento
* Cálculo da conversão
    * Cálculo da métrica de interesse entre os grupos


In [ ]:
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
import statsmodels.stats as st

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Comunidade DS/AB Test/ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [ ]:
# Design Experimento
# nível de confiança
confidence_lvl = 0.95

# nível de significância
significance_lvl = 0.05

# conversões das páginas - real e esperado respectivamente
pg1 = 0.13
pg2 = 0.15

# tamanho do efeito
effect_size = st.proportion.proportion_effectsize(pg1, pg2)

# poder estatístico
power = 0.8

# sample size
n_sample = (
    st.power.NormalIndPower()
    .solve_power(
        effect_size,
        power=power,
        alpha=significance_lvl
    )
)

n_sample = math.ceil(n_sample)
n_sample

4720

### **Análise dos dados**

In [ ]:
# Tamanho tabela
df.shape

(294478, 5)

In [ ]:
# Check null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [ ]:
# Conferir se usuário presente nos 2 grupos
df.groupby(['group', 'landing_page']).agg({'user_id': 'count'})

user_id
group     landing_page         
control   new_page         1928
          old_page       145274
treatment new_page       145311
          old_page         1965

In [ ]:
# Usuários em 2 grupos
dup_users = df.groupby('user_id', as_index=False).agg({'group': 'count'}).query('group > 1')['user_id'].values.tolist()
df = df[~df.user_id.isin(dup_users)]
df.shape

(286690, 5)

### **Amostragem**

In [ ]:
# Amostragem aleatória dos grupos de Controle e Tratamento

# Controle
df_control = df[df.group == 'control'].sample(n_sample, random_state=42)

# Tratamento
df_treatment = df[df.group == 'treatment'].sample(n_sample, random_state=42)

# Concat tabelas
df_ab = pd.concat([df_control, df_treatment], axis=0, ignore_index=True)
df_ab.head()

,user_id,timestamp,group,landing_page,converted
0,663037,2017-01-05 02:13:38.063443,control,old_page,0
1,932826,2017-01-13 04:50:06.941679,control,old_page,1
2,922999,2017-01-04 07:19:23.813317,control,old_page,0
3,857826,2017-01-19 02:27:37.678241,control,old_page,0
4,762926,2017-01-18 17:50:59.261406,control,old_page,0


### **Cálculo das métricas de interesse**

In [ ]:
# Conversão

# Conversão controle
conversion_rate_control = round(df_control[df_control.converted == 1]['converted'].count() / df_control.shape[0], 4)

# Conversão Tratamento
conversion_rate_treatment = round(df_treatment[df_treatment.converted == 1]['converted'].count() / df_treatment.shape[0], 4)

print(f'Conversão Controle: {conversion_rate_control}')
print(f'Conversão Tratamento: {conversion_rate_treatment}')

Conversão Controle: 0.1155
Conversão Tratamento: 0.129
